In [1]:
from forexfactory_scraper import fetch_and_parse_xml, save_events

# Fetch and filter events
events = fetch_and_parse_xml(
    currency_filter="USD",
    impact_filter="High",
    user_timezone="Asia/Tehran"
)


In [2]:
events

[{'title': 'FOMC Meeting Minutes',
  'currency': 'USD',
  'date': '12-30-2025',
  'time': '22:26',
  'impact': 'High',
  'forecast': None,
  'previous': None,
  'url': 'https://www.forexfactory.com/calendar/304-us-fomc-meeting-minutes',
  'sessions': ['New York']},
 {'title': 'Unemployment Claims',
  'currency': 'USD',
  'date': '12-31-2025',
  'time': '16:56',
  'impact': 'High',
  'forecast': '215K',
  'previous': '214K',
  'url': 'https://www.forexfactory.com/calendar/11-us-unemployment-claims',
  'sessions': ['London', 'New York', 'Golden Time']}]

In [27]:
from os import mkdir, path, listdir
from shutil import rmtree
import time
from datetime import datetime
from json import load, dumps, loads
from socket import gethostbyname, gethostname
from random import randint
from uuid import uuid1
from re import sub
from sqlite3 import connect

from psutil import cpu_percent, virtual_memory

import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from requests import post

In [7]:
def initialize_driver():
    """initialize Chrome web driver with configs"""

    options = Options()

    options.add_argument("--disable-notifications")
    options.add_argument('--v=1')
    options.add_argument('--start-maximized')

    return webdriver.Firefox(options=options)

driver = initialize_driver()

In [8]:
events

[{'title': 'FOMC Meeting Minutes',
  'currency': 'USD',
  'date': '12-30-2025',
  'time': '22:26',
  'impact': 'High',
  'forecast': None,
  'previous': None,
  'url': 'https://www.forexfactory.com/calendar/304-us-fomc-meeting-minutes',
  'sessions': ['New York'],
  'text': "It's a detailed record of the FOMC's most recent meeting, providing in-depth insights into the economic and financial conditions that influenced their vote on where to set interest rates;"},
 {'title': 'Unemployment Claims',
  'currency': 'USD',
  'date': '12-31-2025',
  'time': '16:56',
  'impact': 'High',
  'forecast': '215K',
  'previous': '214K',
  'url': 'https://www.forexfactory.com/calendar/11-us-unemployment-claims',
  'sessions': ['London', 'New York', 'Golden Time']}]

In [ ]:
for index, event in enumerate(events):
    if not event.get("text", None):
        driver.get(event['url'])
        time.sleep(3)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        news_text = soup.find('p', class_="calendar-event__description").get_text()
        events[index]['text'] = news_text


In [10]:
events

[{'title': 'FOMC Meeting Minutes',
  'currency': 'USD',
  'date': '12-30-2025',
  'time': '22:26',
  'impact': 'High',
  'forecast': None,
  'previous': None,
  'url': 'https://www.forexfactory.com/calendar/304-us-fomc-meeting-minutes',
  'sessions': ['New York'],
  'text': "It's a detailed record of the FOMC's most recent meeting, providing in-depth insights into the economic and financial conditions that influenced their vote on where to set interest rates;"},
 {'title': 'Unemployment Claims',
  'currency': 'USD',
  'date': '12-31-2025',
  'time': '16:56',
  'impact': 'High',
  'forecast': '215K',
  'previous': '214K',
  'url': 'https://www.forexfactory.com/calendar/11-us-unemployment-claims',
  'sessions': ['London', 'New York', 'Golden Time'],
  'text': "Although it's generally viewed as a lagging indicator, the number of unemployed people is an important signal of overall economic health because consumer spending is highly correlated with labor-market conditions.  Unemployment is 

In [ ]:
pr = """
تو یک تحلیلگر ارشد بازار مالی هستی. رویداد خبری زیر را تحلیل کن و خروجی JSON ارائه بده.

**خروجی موردنظر (فقط JSON):**
{{
  "summary": "خلاصه ۱-۲ جمله‌ای رویداد و پیامد بازار",
  "impact": "High | Medium | Low | None",
  "direction": "Bullish | Bearish | Neutral",
  "confidence": "High | Medium | Low" 
}}

**راهنمای تحلیل:**
- داده اقتصادی بهتر از پیش‌بینی → معمولاً صعودی برای ارز مرتبط
- داده بدتر از پیش‌بینی → معمولاً نزولی برای ارز مرتبط
- بیانیه‌های مرکزی: تأکید بر تورم/رشد → صعودی، تأکید بر ریسک‌ها → نزولی
- در صورت ابهام زیاد، جهت‌گیری خنثی انتخاب کن

**رویداد برای تحلیل:**
{event_data}

**توجه:** سطح تاثیر را از داده ورودی بگیر (تغییر نده).
فقط JSON خالص برگردان.
"""

from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("API_KEY")
client = OpenAI(api_key=api_key)
resp = client.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user","content":pr}])
print(resp.choices[0].message.content)

```json
{
  "summary": "داده‌های اقتصادی جدید نشان داد که رشد اقتصادی بهتر از پیش‌بینی‌ها بوده و این امر می‌تواند موجب تقویت ارز مربوطه شود.",
  "impact": "High",
  "direction": "Bullish",
  "confidence": "High"
}
```


In [37]:


results = []
for event in events:
    formatted_prompt = pr.format(event_data=event)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": formatted_prompt}],
        max_tokens=300,
        temperature=0.0,
    )
    results.append(response.choices[0].message.content)  

In [38]:
results

['```json\n{\n  "summary": "Minutes from the FOMC meeting provide detailed insights into the economic conditions influencing interest rate decisions, which may lead to market volatility.",\n  "impact": "High",\n  "direction": "Neutral",\n  "confidence": "Medium"\n}\n```',
 '```json\n{\n  "summary": "تعداد درخواست\u200cهای بیکاری به 215K افزایش یافته است که بالاتر از پیش\u200cبینی 214K است. این افزایش ممکن است نشان\u200cدهنده ضعف در بازار کار باشد و به تبع آن بر روی دلار تأثیر منفی بگذارد.",\n  "impact": "High",\n  "direction": "Bearish",\n  "confidence": "Medium"\n}\n```']

In [39]:
from typing import List

def convert_str_list_to_json(str_list: List[str]) -> List[dict]:
    """
    Convert a list of JSON strings to a list of dicts.
    Invalid JSON strings will raise json.JSONDecodeError.
    """
    return [json.loads(s) for s in str_list]

cleaned = []
for s in results:
    # If string contains ```json ... ```, extract inner JSON
    if s.strip().startswith("```"):
        # find first newline after opening fence
        parts = s.split("```", 2)
        # parts might be ['', 'json\n{...}\n', ''] or similar; safer to strip the fence
        inner = s.strip().lstrip("`").strip()
        # Remove language token like json at start
        if inner.lower().startswith("json"):
            inner = inner[4:].lstrip()
        # Remove any leading/trailing backticks and whitespace
        inner = inner.strip("` \n")
        cleaned.append(inner)
    else:
        cleaned.append(s)

try:
    result = convert_str_list_to_json(cleaned)
except json.JSONDecodeError as e:
    raise SystemExit(f"Invalid JSON in input: {e}")

In [40]:
result

[{'summary': 'Minutes from the FOMC meeting provide detailed insights into the economic conditions influencing interest rate decisions, which may lead to market volatility.',
  'impact': 'High',
  'direction': 'Neutral',
  'confidence': 'Medium'},
 {'summary': 'تعداد درخواست\u200cهای بیکاری به 215K افزایش یافته است که بالاتر از پیش\u200cبینی 214K است. این افزایش ممکن است نشان\u200cدهنده ضعف در بازار کار باشد و به تبع آن بر روی دلار تأثیر منفی بگذارد.',
  'impact': 'High',
  'direction': 'Bearish',
  'confidence': 'Medium'}]

In [ ]:
{
  'title': 'Federal Funds Rate',
  'currency': 'USD',
  'date': '10-29-2025',
  'time': '21:26',
  'impact': 'High',
  'forecast': '4.00%',
  'previous': '4.25%',
  'url': 'https://www.forexfactory.com/calendar/1-us-federal-funds-rate',
  'sessions': [
    'New York'
  ],
  'text': 'Short term interest rates are the paramount factor in currency valuation - traders look at most other indicators merely to predict how rates will change in the future;The rate decision is usually priced into the market, so it tends to be overshadowed by the FOMC Statement, which is focused on the future;'
}

In [ ]:
{
  'summary': 'The Federal Funds Rate is set to decrease from 4.25% to 4.00%, indicating a potential easing of monetary policy which may weaken the USD.',
  'impact': 'High',
  'direction': 'Bearish'
}

{'summary': 'The Federal Funds Rate is set to decrease from 4.25% to 4.00%, indicating a potential easing of monetary policy which may weaken the USD.',
 'impact': 'High',
 'direction': 'Bearish'}

In [26]:
for result in results:
    print(result)

```json
{
  "summary": "Minutes from the FOMC meeting provide detailed insights into the economic conditions influencing interest rate decisions, which could lead to market volatility.",
  "impact": "High",
  "direction": "Neutral"
}
```
```json
{
  "summary": "تعداد درخواست‌های بیکاری به 215K افزایش یافته است که بالاتر از پیش‌بینی 214K است. این افزایش ممکن است نشان‌دهنده ضعف در بازار کار باشد و بر احساسات بازار تأثیر منفی بگذارد.",
  "impact": "High",
  "direction": "Bearish"
}
```


In [ ]:
import json

# Convert all results to JSON/dictionaries
json_results = [json.loads(result) for result in results]

# Now you have a list of dictionaries
for i, result in enumerate(json_results):
    print(f"Event {i+1}:")
    print(f"  Summary: {result['summary']}")
    print(f"  Impact: {result['impact']}")
    print(f"  Direction: {result['direction']}")
    print()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)